<a href="https://colab.research.google.com/github/AMEYABHAVE/Cristl/blob/main/project2_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.6.0


In [2]:
from google.colab import drive
import pandas as pd
file_location = '/content/drive/MyDrive/accelerometer.csv'
accelerometer =pd.read_csv(file_location)
accelerometer.drop(columns=['time','repetition','uuid'], inplace=True)
accsitting= accelerometer[accelerometer['activity'] == 'CHAIR'].head(31668)
accstanding= accelerometer[accelerometer['activity'] == 'STANDING']
acclying= accelerometer[accelerometer['activity'] == 'LYING_UP']
acclying= acclying.append(accelerometer[accelerometer['activity'] == 'LYING_SIDE'].head(520))
acclying.drop(columns=['activity'], inplace=True)
acclying['activity']='LYING_UP'
accdataset = pd.DataFrame()
accdataset = accdataset.append([accsitting, accstanding, acclying])

file_location1 = '/content/drive/MyDrive/gyroscope.csv'
gyroscope =pd.read_csv(file_location1)
gyroscope.drop(columns=['time','repetition','uuid'], inplace=True)
gyrsitting= gyroscope[gyroscope['activity'] == 'CHAIR'].head(9875)
gyrstanding= gyroscope[gyroscope['activity'] == 'STANDING'].head(9875)
gyrlying= gyroscope[gyroscope['activity'] == 'LYING_UP'].head(9875)
gyrdataset = pd.DataFrame()
gyrdataset = gyrdataset.append([gyrsitting, gyrstanding, gyrlying])

In [3]:
accdataset.info()
gyrdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95004 entries, 0 to 5319
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      95004 non-null  object 
 1   acc_x     95004 non-null  float64
 2   acc_y     95004 non-null  float64
 3   acc_z     95004 non-null  float64
 4   activity  95004 non-null  object 
dtypes: float64(3), object(2)
memory usage: 4.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29625 entries, 0 to 89378
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      29625 non-null  object 
 1   gyr_x     29625 non-null  float64
 2   gyr_y     29625 non-null  float64
 3   gyr_z     29625 non-null  float64
 4   activity  29625 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1.4+ MB


In [4]:
accdataset['activity'].value_counts()

LYING_UP    31668
CHAIR       31668
STANDING    31668
Name: activity, dtype: int64

In [5]:
gyrdataset['activity'].value_counts()

CHAIR       9875
STANDING    9875
LYING_UP    9875
Name: activity, dtype: int64

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
label=LabelEncoder()
accdataset['label']=label.fit_transform(accdataset['activity'])
print(accdataset.head())
print(label.classes_)

            name     acc_x     acc_y     acc_z activity  label
0  accelerometer -0.414825  7.590469  6.235336    CHAIR      0
1  accelerometer -0.337662  7.645508  6.125275    CHAIR      0
2  accelerometer -0.393295  7.605423  6.231155    CHAIR      0
3  accelerometer -0.469269  7.592865  6.281403    CHAIR      0
4  accelerometer -0.418411  7.633530  6.317307    CHAIR      0
['CHAIR' 'LYING_UP' 'STANDING']


In [7]:
gyrdataset['label']=label.fit_transform(gyrdataset['activity'])
print(gyrdataset.head())
print(label.classes_)

        name     gyr_x     gyr_y     gyr_z activity  label
0  gyroscope -0.309540  7.679596  6.142029    CHAIR      0
1  gyroscope -0.530884  7.897354  5.765747    CHAIR      0
2  gyroscope -0.608643  8.231140  5.266846    CHAIR      0
3  gyroscope -0.570953  8.260452  5.229172    CHAIR      0
4  gyroscope -0.816223  8.244308  5.284195    CHAIR      0
['CHAIR' 'LYING_UP' 'STANDING']


In [8]:
AX = accdataset[['acc_x','acc_y','acc_z']]
AY = accdataset['label']
scaler = StandardScaler()
AX = scaler.fit_transform(AX)
Ascaled_X = pd.DataFrame(data=AX, columns=['acc_x','acc_y','acc_z'])
Ascaled_X['label'] = AY.values
Ascaled_X

,acc_x,acc_y,acc_z,label
0,-0.407332,0.535859,0.414297,0
1,-0.342817,0.557656,0.395899,0
2,-0.389331,0.541781,0.413598,0
3,-0.452851,0.536808,0.421998,0
4,-0.410330,0.552913,0.427999,0
...,...,...,...,...
94999,-0.532368,1.405810,-0.635258,1
95000,-0.584891,1.400359,-0.640058,1
95001,-0.561876,1.406281,-0.642660,1
95002,-0.572389,1.409841,-0.661458,1


In [9]:
GX = gyrdataset[['gyr_x','gyr_y','gyr_z']]
GY = gyrdataset['label']
scaler = StandardScaler()
GX = scaler.fit_transform(GX)
Gscaled_X = pd.DataFrame(data=GX, columns=['gyr_x','gyr_y','gyr_z'])
Gscaled_X['label'] = GY.values
Gscaled_X

,gyr_x,gyr_y,gyr_z,label
0,-0.511514,0.743974,0.380241,0
1,-0.751855,0.819761,0.323345,0
2,-0.836288,0.935931,0.247909,0
3,-0.795364,0.946133,0.242213,0
4,-1.061684,0.940514,0.250533,0
...,...,...,...,...
29620,0.393923,1.434373,-0.432256,1
29621,0.417320,1.446039,-0.434066,1
29622,0.326331,1.445205,-0.435876,1
29623,0.383524,1.451038,-0.441668,1
